## Terminology

- **Trial**: A single occurrence with an outcome that is uncertain until we observe it.
For example, rolling a single die.
- **Outcome**: A possible result of a trial; one particular state of the world. What Laplace calls a case.
For example: 4.
- **Sample Space**: The set of all possible outcomes for the trial.
For example, $\{1, 2, 3, 4, 5, 6\}$.
- **Event**: A subset of the sample space, a set of outcomes that together have some property we are interested in.
For example, the event "odd die roll" is the set of outcomes $\{1,3,5\}$.
- **Probability**: As Laplace said, the probability of an event with respect to
  a sample space is the "number of favorable cases" (outcomes from the sample
  space that are in the event) divided by the "number of all the cases" in the
  sample space (assuming "nothing leads us to expect that any one of these cases
  should occur more than any other"), so $P \in [0,1]$.
  For example, the probability of an odd die roll is $3/6 = 1/2$.


## Introduction

### Dice rolling implementation

In [59]:
from fractions import Fraction

def P(event, space):
    "The probability of an event, given a sample space."
    return Fraction(cases(favorable(event, space)),cases(space))

favorable = set.intersection # Outcomes that are in the event and in the sample space
cases     = len              # The number of cases is the length, or size, of a set

D     = {1, 2, 3, 4, 5, 6} # a sample space
even  = {   2,    4,    6} # an event
# NB: intersected with sample space to get favorable outcomes
odd   = {1, 3, 5, 7, 9, 11, 13}
prime = {2, 3, 5, 7, 11, 13}

In [60]:
P(even, D)

Fraction(1, 2)

In [61]:
P(odd, D)

Fraction(1, 2)

In [62]:
P((even | prime), D) # The probability of an even or prime die roll

Fraction(5, 6)

In [63]:
P((odd & prime), D) # The probability of an odd prime die roll

Fraction(1, 3)

### Card problems

In [64]:
import itertools
import random

def combos(items, n):
    "All combinations of n items; each combo as a space-separated str."
    return list(set(map(' '.join, itertools.combinations(items, n))))

suits = u'♥♠♦♣'
ranks = u'AKQJT98765432'
deck  = [r + s for r in ranks for s in suits] # 52
# sample space of all 5-card combinations from deck
Hands = combos(deck, 5) # 2598960

In [65]:
random.sample(Hands, 7)


['K♥ Q♥ Q♠ J♠ 9♦',
 'Q♥ 7♦ 5♠ 3♣ 2♥',
 'K♠ Q♠ 9♣ 6♦ 4♣',
 'A♣ K♦ J♥ 6♥ 5♦',
 'Q♠ 7♠ 4♠ 3♦ 2♣',
 'A♠ J♠ T♠ 5♥ 4♣',
 'A♠ A♦ 9♥ 7♦ 6♠']

In [66]:
random.sample(deck, 7)

['K♣', '6♦', 'J♣', '6♣', '6♥', 'J♥', '9♠']

In [67]:
# 5 cards of the same suit
flush = {hand for hand in Hands if any(hand.count(suit) == 5 for suit in suits)}
P(flush, Hands)

Fraction(33, 16660)

In [68]:
four_kind = {hand for hand in Hands if any(hand.count(rank) == 4 for rank in ranks)}
P(four_kind, Hands) # e.g. 4 kings

Fraction(1, 4165)

## Urn Problems

> *An urn contains 6 blue, 9 red, and 8 white balls.  We select 6 balls at random. What is the probability of each of these  outcomes:*
> 
> - *All balls are red*.
> - *3 are blue, and 1 is red, and 2 are white, *.
> - *Exactly 4 balls are white*.

In [69]:
def balls(color, n):
    "A set of n numbered balls of the given color."
    return {color + str(i)
            for i in range(1, n + 1)}

urn = list(balls('B', 6) | balls('R', 9) | balls('W', 8))

U6 = combos(urn, 6) 
"""sample space (we select 6 random balls)"""

random.sample(U6, 1)

['R6 W4 R5 B2 W7 R4']

In [70]:
def select(color, n, space=U6):
    """
    The subset of the sample space with exactly `n` balls of given `color`.
    Example: `select('R', 6)` is the event of picking 6 red balls from the urn
    """
    return {s for s in space if s.count(color) == n}

In [71]:
# *All balls are red*.
P(select('R', 6), U6) 

Fraction(4, 4807)

In [72]:
# *3 are blue, and 1 is red, and 2 are white, *.
P(select('B', 3) & select('R', 1) & select('W', 2), U6)

Fraction(240, 4807)

In [73]:
# 4 balls (only) are white
P(select('W', 4), U6)


Fraction(350, 4807)

In [74]:
# TODO: what about probability that 4 or more are white? 


### Verifying urn calculations with arithmetic

Let's verify the first question of drawing exactly 6 red balls.

We need to first answer _how many ways can I choose 6 out of 9 red balls?_

If we care about the order in which they're drawn, the number of ways of choosing $k$ out of $n$ items is:

$$
\frac{n!}{(n - k)!}
$$

But if we don't care about the *order* of the six drawn balls, we can divide
that product by the number of permutations of 6 things, i.e. 6:

$$
\frac{n!}{(n - k)! \times k!}
$$


In [75]:
from math import factorial

def choose(n, k):
    "Number of ways to choose k items from a list of n items without taking order into account."
    return factorial(n) // (factorial(n - k) * factorial(k))

choose(9, 6)

84

In [91]:
N = len(U6)

# `P` computes a ratio and `choose` computes a count,
# so we can multiply the probability back by the denominator `cases(space)`, i.e. len(U6) 
# so that both are counts.
print(N * P(select('R', 6), U6) == choose(9, 6))
# which is just:
print(len(favorable(select('R', 6), U6)) == choose(9, 6))

True
True


In [92]:
# 3 are blue, and 1 is red, and 2 are white

N * P(select('B', 3) & select('W', 2) & select('R', 1), U6) == choose(6, 3) * choose(9, 1) * choose(8, 2) 

True

In [93]:
# 4 balls (only) are white
N * P(select('W', 4), U6) == choose(8, 4) * choose(6 + 9, 2)  # (6 + 9 non-white balls)

True


## References

- Peter Norvig's [ipynb](https://github.com/norvig/pytudes/blob/main/ipynb)
-